In [2]:
import nlp_toolkit


ModuleNotFoundError: No module named 'torch'

In [2]:
import pandas as pd
from nlp_toolkit import (
    set_proxy,
    SentenceEmbedder,
    make_chroma,
    insert_df_collection
)

set_proxy(33210)

ModuleNotFoundError: No module named 'torch'

In [3]:
pwd

'd:\\pycharm\\nlp_toolkit\\nlp_toolkit'

In [10]:
description = pd.read_feather(r'D:\pycharm\nlp_toolkit\appvec_incomplete.fth')
description = description.sort_values('sim',ascending=False).reset_index(drop=True)


NameError: name 'pd' is not defined

In [5]:
description = description[['package_name','data_country_id','app_name','score_old','installs_old','genre_old','title','summary','comments','description','url']]
description['index'] = description.index

In [6]:
sentences = description.summary.tolist()[:2000]
embedder = SentenceEmbedder(batch_size=128)

# this is the object used by chromadb
chroma_embedder = make_chroma(embedder)
chroma_embedder(['what is the fuck']).shape

cpu
Embedding Batch: 1......Done.


(1, 384)

In [7]:
embeded_sentences = embedder(sentences)

Embedding Batch: 1......

KeyboardInterrupt: 

In [35]:
import chromadb

chroma_client = chromadb.PersistentClient(path='./chroma_storage')
chroma_client.delete_collection("app")
collection = chroma_client.create_collection(name="app", embedding_function=chroma_embedder)

insert_df_collection(
    collection = collection,
    embeddings = embeded_sentences.tolist(), 
    df = description[:32], 
    doc_col = 'summary', 
    id_col = 'index', 
    meta_cols = None
)

Successful added to collection


In [37]:
results = collection.query(
    query_texts=["loan,credit"],
    n_results=20,
)

results['metadatas']

Embedding Batch: 1......Done.


[[{'app_name': 'CreditKes',
   'comments': '',
   'data_country_id': 254,
   'description': 'CreditKes is a secure and convenient loan platform for Kenya Citizens.',
   'genre_old': '"FINANCE"',
   'installs_old': 399926.0,
   'package_name': 'com.creditkes.app',
   'score_old': 0.0,
   'title': 'CreditKes',
   'url': 'https://play.google.com/store/apps/details?id=com.creditkes.app&hl=en&gl=us'},
  {'app_name': 'Creditmoja',
   'comments': "The loan was issued pretty fast. However, look carefully at the repayment options. One option is much more costly. Secondly, the harassment via text starts two days before the loan due date. The loan processing is very efficient. The problem is their interest rate. It's extremely high. The first time they charged 600 for a loan of 2k so I received 1400. They promised to increase the limit and reduce interest the second borrowing which they increased to 3600 and charged 600. I got some cash and repaid the second one earlier but they have reduced my l